In [1]:
import pandas as pd
from pathlib import Path

# Paths
raw_csv   = Path("../data/raw/accepted_2007_to_2018Q4.csv")
trim_csv  = Path("../data/raw/accepted_2017.csv")
trim_pq   = Path("../data/raw/accepted_2017.parquet")

# 1) Load the full LendingClub file
df = pd.read_csv(raw_csv, low_memory=False)

# 2) Keep only 2017 vintages (fully-seasoned loans)
df_2017 = df[df["issue_d"].str.contains("2017", na=False)].copy()
print("✅ Trimmed shape:", df_2017.shape)

# 3) Save a lightweight CSV and a compact Parquet
df_2017.to_csv(trim_csv, index=False)

# Optimise string columns to category before Parquet
for col in df_2017.select_dtypes("object"):
    df_2017[col] = df_2017[col].astype("category")

df_2017.to_parquet(trim_pq, compression="snappy")
print("✅ Saved parquet OK:", df_2017.shape)

✅ Trimmed shape: (443579, 151)
✅ Saved parquet OK: (443579, 151)


## 📝 Why re-trim to 2017 only?

After analysing default rates, I found late-2018 loans show < 1 % charge-off, while 2017 loans are ~6–11 %.  
This gap exists because the public LendingClub snapshot was taken in early 2019, so many 2018 loans hadn’t had time to default.

By limiting the dataset to 2017 only, every loan has ≥ 24 months of observed performance, giving a consistent default rate across the cohort.

**Next steps (in 01_eda and beyond)**

1. Clean & engineer features on the 2017 data.
2. Re-run feature selection (leak-free) and save `feature_lists_v3.pkl`.
3. Perform a time-based split 70/30 split (train on loans ≤ 2017-09-01; test on loans > 2017-09-01) and save `train_v2` / `test_v2`.
4. Fit baseline Logistic Regression again, then return to trying Gradient Boosting / XGBoost.